In [10]:
import numpy as np
import random

def deriv(f, x, h=0.00001):
    return (f(x+h) - f(x-h))/(2*h)

def s_deriv(f, x, h=0.00001):
    return (f(x+h) - 2.0*f(x) + f(x-h))/(h*h)

def parabolic_iteration(f, l, r, e=1.0e-20):
    count = 0
    u = l
    v = (r-l)/2.0
    w = r

    while abs(u-w) > e:
        fu = f(u)
        fv = f(v)
        fw = f(w)
        p = (v - u)**2 * (fv - fw) - (v - w)**2 * (fv - fu)
        q = (v - u) * (fv - fw) - (v - w) * (fv - fu)
        if q == 0:
            break
        next = v - p/(2*q)
        
        u = w
        w = v
        v = next

        count+=1

    return v, count

def netwon_optimization(f, l, r, e=1.0e-20):
    count = 0
    x = (r-l)/2.0
    step = 1.0
    while abs(step) > e and count <10:
        step = deriv(f,x)/s_deriv(f, x)
        x -= step
        count+=1
    return x, count



The above functions perform optimization in 1 dimension in two different ways, using Newtons method and using interpolation by a parabola and taking its minima to obtain new esimates. Lets test the number of steps needed to obtain a certain accuracy for each method

In [11]:
def f(x):
    return 0.5 - x*np.exp(-x*x)

def f2(x):
    return x*x - x

def f3(x):
    return x*x*x - x*x - 2.0*x

print('(solution, iterations)')
print(netwon_optimization(f, 0, 1))
print(parabolic_iteration(f, 0, 1))
print(netwon_optimization(f2, 0, 3))
print(parabolic_iteration(f2, 0, 3))
print(netwon_optimization(f3, 0, 3))
print(parabolic_iteration(f3, 0, 3))

(solution, iterations)
(0.7071067812116412, 6)
(0.7071067840797851, 13)
(0.4999999999999997, 3)
(0.5, 2)
(1.2152504370068424, 5)
(1.2152504394041972, 9)


Newtons method appears to be quicker in general, of course this comes at the cost of extra error and computation needed for computing the derivative. Both of these methods may also fail to converge in some instances, and newtons method may instead converge to an inflection point or maximum...

In [3]:
# generate some examples of failures to converge
print('(solution, iterations)')
print(netwon_optimization(np.cos, 2, 4))
print(parabolic_iteration(np.cos, 0, 3))
print(parabolic_iteration(np.cos, 2, 4))


(solution, iterations)
(-4.4948413616347285e-12, 5)
(-6.283185307201398, 11)
(3.1415926536331837, 8)


There are methods that have guarenteed convergence but this generally comes at the cost of a slower convergence rate. Below is an example of this know as the golden section method

In [4]:
# implement golden section method
def golden_section(f, l, r, e=1.0e-10):
    count = 0
    t = (np.sqrt(5) - 1.0)/2.0
    x1 = l + (1-t)*(r-l)
    f1 = f(x1)
    x2 = l + t*(r-l)
    f2 = f(x2)

    while r-l > e:
        if f1 > f2:
            l = x1
            x1 = x2
            f1 = f2
            x2 = l + t*(r-l)
            f2 = f(x2)
        else:
            r = x2
            x2 = x1
            f2 = f1
            x1 = l + (1-t)*(r-l)
            f1 = f(x1)
        count += 1

    return l, r, count


print('(left, right, iterations)')
print(golden_section(np.cos, 1, 4))
print(golden_section(f2, 0, 3))
print(golden_section(f3, 0, 3))

(left, right, iterations)
(3.1415926430465917, 3.1415926431124714, 51)
(0.4999999962345232, 0.4999999963004029, 51)
(1.215250425468034, 1.2152504255339138, 51)


We can combine techniques to achieve guaranteed convergence while hopefully increasing the speed substantially. To do this we simply run the golden section method to some tolerence level then apply parabolic iterations

In [13]:
# combined method
def combined_optimization(f, l, r, e=1.0e-20):
    
    l, r, g_count = golden_section(f, l, r, 1.0e-3)
    x, p_count = parabolic_iteration(f, l, r, e=1.0e-20)
    return x, g_count, p_count


print('(solution, golden iterations, parabolic iterations)')
print(combined_optimization(np.cos, 2, 4))
print(combined_optimization(f2, 0, 3))
print(combined_optimization(f3, 0, 3))

(solution, golden iterations, parabolic iterations)
(3.1415926535886403, 16, 6)
(0.5000000000000473, 17, 3)
(1.2152504346241997, 17, 6)


These are much more promising results, we obtain twice the accuracy as the golden section method in roughly half the number of steps while still benefiting from the improved convergence properties.